

   ##Julia and the World

### Calling C Programs

* Seamless calls to C programs is a major feature 
* Zero overhead 
* No compilation / pre-processing required
 * Directly call shared libraries

## What does the compiler need to know to call a C function

* The library
* The name
* The number and types of arguments
* The return type
* The argument values

## `ccall`

`ccall( (:name, "lib"), return_type, (arg1_type, arg2_type...), arg1, arg2...)`

In [29]:
ccall( (:clock, :libc), Int32, ())

9758500

In [10]:
ccall( (:clock, "libc"), Cint, ())

9049174

### Aliases for C types

* Cint, Cuint
* Clong, Culong
* Cchar
* Csize_t
* ... etc 
http://docs.julialang.org/en/release-0.3/manual/calling-c-and-fortran-code/#type-correspondences

In [9]:
path = ccall( (:getenv, :libc), Ptr{Cchar}, (Ptr{Cchar},), "SHELL")
bytestring(path)

"/bin/bash"

### Ptr{T} : The pointer type

* `Ptr{Cint}` : `int*`
* `Ptr{Unit8}` or `Ptr{Cchar}` : `char*`
* `Ptr{Void}` or `Ptr{None}` : `void*`

### Pointer conversions

* Julia Strings converted to `Ptr{Cchar}`
* Array{T} converted to `Ptr{T}`
* Other values can be converted by &
 * Not the same as C & 
 * No writeback for immutables
 * In `v0.4` use Ref{T}

### The Array Trick
* A Julia Array can be used to allocate memory on the Julia side
* The array can be passed as a pointer

In [33]:
hostname = Array(Uint8, 128)
ccall( (:gethostname, :libc), Int32, (Ptr{Uint8}, Uint), 
                                       hostname, length(hostname))
bytestring(convert(Ptr{Uint8}, hostname))

"avik-mb.local"

### Structs
* C structs can be replicated on the Julia side by a composite type
* In 0.3, structs can only be passed by via a pointer
* In 0.4, structs can also be passed by value

In [3]:
type TimeStruct
    sec::Int32
    min::Int32
    hour::Int32
    mday::Int32
    month::Int32
    year::Int32
    wday::Int32
    yday::Int32
    isdst::Int32
    # on some platforms the struct is 14 words, even though 9 are specified
    _10::Int32
    _11::Int32
    _12::Int32
    _13::Int32
    _14::Int32

    TimeStruct() = new(0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    
end
function strptime (date_string)
    tm = TimeStruct()
    r = ccall( (:strptime, :libc), Ptr{Uint8}, (Ptr{Uint8}, Ptr{Uint8}, Ptr{Void}), 
                date_string, "%c", &tm)
    if r==C_NULL; error("Invalid Date"); end
    return tm
end


strptime (generic function with 1 method)

In [5]:
strptime("Fri 26 Jun 22:18:16 2015")

TimeStruct(16,18,22,26,5,115,5,0,0,0,0,0,0,0)

###unsafe_load

* Given a value `x` of type `Ptr{T}` where `T` is a julia type:
* `unsafe_load(x)` will copy the data referenced by the pointer 
* into a julia object of type `T`


* when the pointer of type `Ptr{T}` points to an array of elements `T`
* unafe_pointer_to_array(x, dims, [own]) creats an Array{T, N}
* where N = length(dims)
* the underlying data is not copied, and is freed by the Julia GC is `own=true`


#### The function is called `unsafe` for a reason... segfaults are to be expected if you lie!

#Exercise: GMP

## Advanced: Passing Julia Functions into C code



### Standard library C qsort routine
* pass a custom comparison function to `qsort`

`void qsort(void *base, size_t nmemb, size_t size,
           int(*compare)(const void *a, const void *b));`

In [47]:
function julia_compare{T}(a_pointer::Ptr{T}, b_pointer::Ptr{T})
    a::T = unsafe_load(a_pointer)
    b::T = unsafe_load(b_pointer)
    println("Sorting $a and $b of type $T") 
    return a < b ? cint(-1) : a > b ? cint(+1) : cint(0)
end
cint(x) = convert(Cint, x)

cint (generic function with 1 method)

In [48]:
const julia_compare_c = cfunction(julia_compare, Cint, (Ptr{Cdouble}, Ptr{Cdouble}))

Ptr{Void} @0x00000001135cfe30

In [49]:
A = [2.2, 1.3, -3.4, 5.6, 2.6]
ccall( (:qsort, :libc) , Void, (Ptr{Cdouble}, Csize_t, Csize_t, Ptr{Void}),
              A, length(A), sizeof(eltype(A)), julia_compare_c)

Sorting 2.2 and 1.3 of type Float64
Sorting 2.2 and -3.4 of type Float64
Sorting 1.3 and -3.4 of type Float64
Sorting 2.2 and 5.6 of type Float64
Sorting 5.6 and 2.6 of type Float64
Sorting 2.2 and 2.6 of type Float64


In [45]:
A

5-element Array{Float64,1}:
 -3.4
  1.3
  2.2
  2.6
  5.6

## Even More Advanced: Calling functions from a different C thread

In [53]:
#Dummy code
real_callback(data) = print(data)

function real_callback_handle(obj::Any)
    work = Base.SingleAsyncWork(real_callback)
    work.handle
end

function callback_other_thread(handle::Ptr{Void})
    ccall(:uv_async_send,Cint,(Ptr{Void},),handle)
end

const callback_other_thread_c = cfunction(callback_other_thread, Cint, (Ptr{Void}, ))


Ptr{Void} @0x00000001135d0620